# dev-multiple shooting

Yuri Shimane, 2021/07/06

In [263]:
using LinearAlgebra
using DataFrames
using Plots
using DifferentialEquations
using BenchmarkTools

In [156]:
plotly()

Plots.PlotlyBackend()

In [161]:
# push!(LOAD_PATH, "../R3BP/src")
# using R3BP
include("../R3BP/src/R3BP.jl")

Main.R3BP

In [27]:
reltol = 1.e-13
abstol = 1.e-13
method = Tsit5()

Tsit5()

In [28]:
# construct halo initial guess
CR3BP_param = R3BP.get_cr3bp_param(10, 299)
lp = 2
Az_km = 260*1e3
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(CR3BP_param.mu, lp, Az_km, CR3BP_param.lstar, northsouth)
#res = R3BP.ssdc_periodic_xzplane(CR3BP_param.mu, guess0.x0, guess0.period, fix="z");

Halo guess Az_km: 260000.0


In [29]:
guess0.fullstates

500×7 Array{Float64,2}:
 1.00734   0.0          -0.00214756  …  0.0108476   0.0          0.0
 1.00734   6.71685e-5   -0.00214731     0.0108465   8.19178e-5   0.00611828
 1.00734   0.000134323  -0.00214656     0.010843    0.000163809  0.0122366
 1.00734   0.000201449  -0.0021453      0.0108373   0.000245647  0.0183548
 1.00734   0.000268532  -0.00214355     0.0108293   0.000327406  0.0244731
 1.00734   0.000335559  -0.0021413   …  0.010819    0.000409059  0.0305914
 1.00734   0.000402514  -0.00213855     0.0108064   0.00049058   0.0367097
 1.00735   0.000469385  -0.00213529     0.0107915   0.000571942  0.042828
 1.00735   0.000536157  -0.00213155     0.0107744   0.00065312   0.0489462
 1.00735   0.000602815  -0.0021273      0.010755    0.000734087  0.0550645
 1.00735   0.000669346  -0.00212256  …  0.0107333   0.000814818  0.0611828
 1.00735   0.000735737  -0.00211733     0.0107094   0.000895287  0.0673011
 1.00735   0.000801972  -0.00211161     0.0106833   0.000975468  0.0734194
 ⋮     

In [348]:
n_data, _ = size(guess0.fullstates)   # number of data in initial guess array
nms = 6   # number of nodes to use
x0s, tofs = [], []

skip_idx = Int(floor(500/(nms-1)))
println("skip_idx: $skip_idx")

extract_idx = []
for i = 1:skip_idx:n_data
    push!(extract_idx, i)
end
# append final node for periodic orbit
push!(extract_idx, 1)

# get x0s
for i = 1:nms
    extract_idx[i]
    push!(x0s, guess0.fullstates[extract_idx[i], 1:6])
end

# get tofs
for i = 1:nms-1
    if i != nms-1
        tof = guess0.fullstates[extract_idx[i+1], 7] - guess0.fullstates[extract_idx[i], 7]
    else
        tof = guess0.period - guess0.fullstates[extract_idx[i], 7]
    end
    push!(tofs, tof)
end

nms, length(x0s), length(tofs)

skip_idx: 100


(6, 6, 5)

In [349]:
x0s

6-element Array{Any,1}:
 [1.0073423859117585, 0.0, -0.002147558386821189, 0.0, 0.010847611831763939, 0.0]
 [1.008074665318409, 0.0047472578736575, -0.00018919196412906297, 0.003490294218367376, 0.0023585089540287983, 0.004957591854174411]
 [1.0100050182103024, 0.002742787725799727, 0.00231666978217853, 0.0010794051685515844, -0.0078333272711956, 0.0024776764868559573]
 [1.0099906048308174, -0.002791091362531503, 0.0023013750890842617, -0.001108276135805258, -0.007767871602637049, -0.00252194347674866]
 [1.008058138759838, -0.004732267034693988, -0.00021951334762114073, -0.0034866968689220546, 0.0024833284914135834, -0.004954001858024531]
 [1.0073423859117585, 0.0, -0.002147558386821189, 0.0, 0.010847611831763939, 0.0]

In [350]:
μ = CR3BP_param.mu
prob_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, vcat(guess0.x0, reshape(I(6), (36,)))[:], guess0.period, (μ));

In [351]:
nms, length(x0s), length(tofs)

(6, 6, 5)

In [364]:
include("../R3BP/src/R3BP.jl")

Main.R3BP

In [365]:
typeof(x0s)

Array{Any,1}

In [366]:
# solve with multiple-shooting
tolDC = 1.e-13
x0_conv, tof_conv = R3BP.multiple_shooting(prob_stm, x0s, tofs, tolDC; periodic=true, reltol=reltol, abstol=abstol);

Multiple-shooting using 6 nodes, periodic: true
Multiple shooting iteration 1, error = 0.006620753837688195
Multiple shooting iteration 2, error = 0.00020983098061856125
Multiple shooting iteration 3, error = 2.187940939081456e-6
Multiple shooting iteration 4, error = 7.381910877005488e-9
Tolerance achieved at iteration 5 with error = 2.213458542523518e-14


In [367]:
# propagate converged result
prob = ODEProblem(R3BP.rhs_cr3bp_sv!, x0_conv[1:6], sum(tof_conv), (μ))
sol = solve(prob, Tsit5(), reltol=reltol, abstol=abstol);
solmsdc = R3BP.sol_to_arrays(sol)

6×287 Array{Float64,2}:
  1.00715       1.00715       1.00715      …   1.00715       1.00715
  0.000193789   0.000217041   0.000247827      0.000140104   0.000193789
 -0.00233766   -0.00233707   -0.00233619      -0.00233876   -0.00233766
  4.89147e-5    5.48104e-5    6.26306e-5       3.53316e-5    4.89147e-5
  0.0113005     0.0112979     0.0112939        0.0113054     0.0113005
  0.000269467   0.000301778   0.000344547  …   0.000194842   0.000269467

In [368]:
idx1, idx2 = 1, 2
ptraj = plot(flip=false, aspect_ratio=:equal, size=(800,650))
plot!(ptraj, solssdc[idx1,:], solssdc[idx2,:], label="SSDC")
plot!(ptraj, solmsdc[idx1,:], solmsdc[idx2,:], label="MSDC")

# plot nodes
for i = 1:nms
    scatter!(ptraj, [x0_conv[(i-1)*6+1:6i][idx1]], [x0_conv[(i-1)*6+1:6i][idx2]], 
        label="MSDC-node $i", marker=(:cross, 3.5, 3.5))
end
ptraj

<!DOCTYPE html>
 
 
 Plots.jl

In [285]:
# solve with single-shooting
res = R3BP.ssdc_periodic_xzplane(CR3BP_param.mu, guess0.x0, guess0.period, fix="z");
solssdc = R3BP.sol_to_arrays(res.sol)

Converged at iteration 5


6×182 Array{Float64,2}:
  1.00726      1.00726      1.00726      …   1.00726      1.00726
  0.0          3.60384e-5   8.41048e-5      -5.1874e-5    1.48224e-12
 -0.00214756  -0.00214748  -0.00214714      -0.0021474   -0.00214756
  0.0          8.71488e-6   2.03451e-5      -1.25453e-5  -4.59358e-12
  0.0109584    0.010958     0.0109564        0.0109576    0.0109584
  0.0          4.66846e-5   0.000108946  …  -6.71974e-5   1.58926e-12

In [286]:
idx1, idx2 = 1,3
ptraj = plot(flip=false, aspect_ratio=:equal, size=(800,650))
plot!(ptraj, solssdc[idx1,:], solssdc[idx2,:], label="SSDC")
plot!(ptraj, solmsdc[idx1,:], solmsdc[idx2,:], label="MSDC")
ptraj

<!DOCTYPE html>
 
 
 Plots.jl

In [259]:
foo = rand(4,4)

4×4 Array{Float64,2}:
 0.46478   0.784674  0.0925251  0.823108
 0.926587  0.149865  0.852336   0.492577
 0.393189  0.405266  0.63609    0.263498
 0.106885  0.567023  0.569524   0.527827

In [272]:
foo[end-1:end, 1:2] = I(2)

2×2 Diagonal{Bool,Array{Bool,1}}:
 1  ⋅
 ⋅  1

In [273]:
foo

4×4 Array{Float64,2}:
 0.46478   0.784674  0.0925251  0.823108
 0.926587  0.149865  0.852336   0.492577
 1.0       0.0       0.63609    0.263498
 0.0       1.0       0.569524   0.527827